# Packages

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import pickle

**"bairro_group_type_Manhattan", "longitude", "room_type_type_Entire home/apt" e "room_type_type_Private room"**

# Carregando e preparando dataset para modelos

Apesar de no EDA ter sido feito o filtro para preços entre 0 e 300. Percebeu-se que os modelos tiveram resultados melhores reduzindo o intervalo entre 0 e 230

Tirando isso, o mesmo pre-processamento feito no EDA será feito aqui

In [4]:
df = pd.read_csv("../assets/teste_indicium_precificacao.csv")

In [5]:
df_pp = df.copy()
df_pp = df_pp[(df_pp['price'] > 0) & (df_pp['price'] <= 230)]
df_pp = pd.get_dummies(df_pp, columns=['bairro_group'], prefix=['bairro_group_type'], dtype=int)
df_pp = pd.get_dummies(df_pp, columns=['room_type'], prefix=['room_type_type'], dtype=int)
df_pp.loc[:, 'ultima_review_timestamp'] = df_pp['ultima_review'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').timestamp() if pd.notnull(x) else None)
df_pp = df_pp.dropna()


Como o método de correlação usada no EDA foi 'Pearson' e seu método de avaliação leva em consideração a correlação linear entre as características do dataset, também sera usado aqui modelos lineares

Inicialmente foi experimentado utilizar apenas as características que tinham mais correlação com o preço. Mas os resultados foram piores do que usando todas as variáveis

Além disso, será utilizado como métrica de avaliação o Root Mean Squared Error (RMSE). Cuja avaliação consiste em fazer o somatório da subtração do valor preditodo do valor real ao quadrado e posteriormente tirar sua raiz. Comportamento parecido ao realizar se uma distância euclideana entre dois pontos. Decidiu-se por essa métrica por eu ter percebido muitos autores da literatura a utilizando para avaliar modelos de regressão. Tenho pouca experiência com tarefas de regressão, então decidi por confiar na literatura

In [6]:
X = df_pp.drop("price", axis=1)
# X = df_pp[['longitude', 'room_type_type_Entire home/apt', 'room_type_type_Private room', 'bairro_group_type_Manhattan']]
y = df_pp.price

X_norm = X.select_dtypes(exclude=["object"])
X_norm = X_norm.astype('float')
X_norm = X_norm.iloc[:, 2:]
columns = X_norm.columns
mms = MinMaxScaler()
X_norm = mms.fit_transform(X_norm)
X_norm = pd.DataFrame(X_norm, columns=columns)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

# Declariando modelos para GridSearchCV

Aqui será utilizado validação cruzada. Esse método consiste em dividir o split em K partes e fazer uma iteração em que cada parte vire um conjunto de teste. Isso é importante para escolher os melhores modelos sem viés e com os melhores parâmetros

## LinearRegression

In [7]:
lin_params = {}

## Ridge

In [8]:
ridge_params = {
    'alpha': [0.01],  # Regularization strength
    'solver': ['saga'],  # Solver to use
    'max_iter': [1000]  # Maximum number of iterations
}

## SGDRegressor

In [9]:
sgd_params = {
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],  # Loss function to be optimized
    'penalty': ['none', 'l2', 'l1', 'elasticnet'],  # Regularization penalty
    'alpha': [0.1],  # Regularization strength
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],  # Learning rate schedule
    'max_iter': [1000],  # Maximum number of iterations
    'tol': [1e-3]  # Tolerance for the stopping criterion
}

# Grid search

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor


In [11]:
model_params = ([LinearRegression(), Ridge(), SGDRegressor()],
                [lin_params, ridge_params, sgd_params])

In [12]:
list_best_models_params = []
for model, params in zip(model_params[0], model_params[1]):
    gs = GridSearchCV(model,
                      param_grid=params,
                      scoring='neg_root_mean_squared_error',
                      )

    gs.fit(X_train, y_train)
    print(f"Best CV results for {model.__class__.__name__}")
    print("Best Score of train set: " + str(gs.best_score_))
    print("Best estimator: " + str(gs.best_estimator_))
    print("Best parameter set: " + str(gs.best_params_))

    store_best_model_configs = {
        'model_name': model.__class__.__name__,
        'best_score': gs.best_score_,
        'best_estimator': gs.best_estimator_,
        'best_params': gs.best_params_
    }

    list_best_models_params.append(store_best_model_configs)

df_best_models_params = pd.DataFrame(list_best_models_params)
df_best_models_params.to_csv('../../assets/best_models_params_cv.csv', index=False)

df_best_models_params

Best CV results for LinearRegression
Best Score of train set: -38.83084344085628
Best estimator: LinearRegression()
Best parameter set: {}
Best CV results for Ridge
Best Score of train set: -38.83083752464188
Best estimator: Ridge(alpha=0.01, max_iter=1000, solver='saga')
Best parameter set: {'alpha': 0.01, 'max_iter': 1000, 'solver': 'saga'}


C:\Users\guimi\miniconda3\envs\sirius\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
140 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\guimi\miniconda3\envs\sirius\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\guimi\miniconda3\envs\sirius\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\guimi\miniconda3\envs\sirius\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraint

Best CV results for SGDRegressor
Best Score of train set: -38.83104540605827
Best estimator: SGDRegressor(alpha=0.1, learning_rate='adaptive',
             loss='squared_epsilon_insensitive', penalty='l1')
Best parameter set: {'alpha': 0.1, 'learning_rate': 'adaptive', 'loss': 'squared_epsilon_insensitive', 'max_iter': 1000, 'penalty': 'l1', 'tol': 0.001}


OSError: Cannot save file into a non-existent directory: '..\..\assets'

In [ ]:
best_model = Ridge(**{'alpha': 0.01, 'max_iter': 1000, 'solver': 'saga'}).fit(X_train, y_train)

In [ ]:
with open('../assets/supervised_best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)


# Conclusão

Inicialmente, tentou-se utilizar apenas as 4 variaveis com maior correlação com o preço para os modelos preditivos. Mas os resultados não foram satisfatórios, com RMSE em torno de 38.83.
Ao utilizar todas as variáveis, o RMSE ficou um pouco melhor, em torno de 35. Mas ainda sim muito alto. Valores bons de RMSE costumam ser abaixo de 10.

Todos os três modelos lineares testados em splits de treino tiveram desempenho relativamente igual. Portanto, foi escolhido um modelo arbitrário dos três